<a href="https://colab.research.google.com/github/mostly-sunny/digital-health-hackathon/blob/main/3.%20coxph_find_best_network_by_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pycox - CoxPH Model
- Network : Test with sets, and find best network and lr by lowest brier score
- Input Variables : G1 ~ G300, Var1 ~ Var10, Treatment
- Output Variables : time, event
- Scaler : MinMaxScaler -> Var1 ~ Var10



In [27]:
pip install pycox

In [28]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn_pandas import DataFrameMapper
import pandas as pd

import torch
import torchvision
import torchtuples as tt

from pycox.models import CoxPH
from pycox.evaluation import EvalSurv

In [29]:
torch.cuda.is_available()

True

In [30]:
np.random.seed(123456)
_ = torch.manual_seed(123456)

- all-in-one.csv 파일은 유전자 변이 유무, 임상 변수, 생존 기간, 사망 여부, 치료 유무가 열로 존재하는 파일
- test-data-treat-and-untreat.csv 파일은 all-in-one의 열은 같은 602개의 데이터.
  - (0번째 행) : 유전자 변이 모두 0, 치료 0
  - (1번째 행) : 유전자 변이 모두 0, 치료 1
  - (2~301번째 행) : 유전자 변이 n-1에만 1, 치료 0
  - (302~601번재 행) : 유전자 변이 n-301에만 1, 치료 1
- pandas 라이브러리에 있는 csv 파일을 DataFrame으로 바꾸어주는 read_csv 함수를 이용하여 파일을 읽어 들임.
- DataFrame은 표를 나타내는 데이터 타입임.

In [31]:
dataset = pd.read_csv('/content/all-in-one.csv')
dataset_for_hr = pd.read_csv('/content/test-data-treat-and-untreat.csv')

- 위에서 읽어들인 dataset 중에서 20%는 검증(_val -> validation)을 위해 sampling 한다.
- 남은 80%의 데이터에서도 20%는 테스트(_test)를 위해 sampling 한다.

In [32]:
dataset_test = dataset.sample(frac=0.3, random_state = np.random.seed(123456))
dataset_train = dataset.drop(dataset_test.index)
dataset_val = dataset_train.sample(frac=0.2, random_state = np.random.seed(123456))
dataset_train = dataset_train.drop(dataset_val.index)

In [33]:
dataset_train

,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,G36,G37,G38,G39,G40,...,G274,G275,G276,G277,G278,G279,G280,G281,G282,G283,G284,G285,G286,G287,G288,G289,G290,G291,G292,G293,G294,G295,G296,G297,G298,G299,G300,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,time,event,Treatment
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,5,1,1,4,6,5,2,1,0,1,57.448331,1,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,3,0,0,2,2,6,3,3,1,44.559284,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,5,5,3,3,5,6,1,2,6,2,68.660751,1,0
7,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,2,0,2,6,7,3,3,1,3,1,53.705056,1,1
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,1,1,4,2,4,7,4,1,0,94.614254,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,5,3,2,4,9,4,3,3,0,1,59.763248,1,0
991,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,3,4,5,2,3,1,1,4,4,2,62.919174,1,0
995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,4,2,1,2,2,2,2,1,4,4,66.591235,1,1
996,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,4,3,4,9,3,6,6,4,2,0,62.986021,0,1


- columns_standardize : 임상변수 - 0~9사이의 값을 표준화
- columns_leave : 유전자 변이 유무 + 치료 유무 - 0과 1로 표현돼 있기 때문에 표준화 필요 없음.
- DataFrameMapper는 pandas DataFrame에서 원하는 열을 뽑아서 리스트로 만들어줌.
- 리스트로 만들때 StandardScaler()가 포함된 열은 표준화를 시킨 뒤, 그리고 None이면 갖고 있는 값을 그대로 넣음.

In [34]:
columns_standardize = ['Var' + str(i) for i in range(1,11)]
columns_leave = ['G' + str(i) for i in range(1,301)]
columns_leave += ['Treatment']

# standardize = [([col], StandardScaler()) for col in columns_standardize]
standardize = [([col], MinMaxScaler()) for col in columns_standardize]

leave = [(col, None) for col in columns_leave]

x_mapper = DataFrameMapper(leave + standardize)

- 위에서 만든 DataFrameMapper로 DataFrame 중 x(입력) 데이터를 모델이 학습할 수 있게끔 리스트 형식으로 바꾸어 준다.



In [35]:
x_train = x_mapper.fit_transform(dataset_train).astype('float32')
x_val = x_mapper.transform(dataset_val).astype('float32')
x_test = x_mapper.transform(dataset_test).astype('float32')
x_for_hr = x_mapper.transform(dataset_for_hr).astype('float32')

- DataFrame (표)에서 Y(출력)데이터인 time(생존기간)과 event(사망여부)를 뽑아 출력 데이터를 추린다.
- 검증(Validation)을 위한 입력-출력 세트 val을 만든다.

In [36]:
get_target = lambda df: (df['time'].values, df['event'].values)
y_train = get_target(dataset_train)
y_val = get_target(dataset_val)

durations_test, events_test = get_target(dataset_test)
val = x_val, y_val

함수 make_net : network을 생성해 리턴하는 함수
- input과 output의 노드 수, 은닉층 수, 은닉층의 노드 수 설정 가능

In [37]:
def make_net(in_features, out_features, hidden, nodes):
  if hidden == 1:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),
            
      torch.nn.Linear(nodes, out_features)
    )
  elif hidden == 2:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),
            
      torch.nn.Linear(nodes, out_features)
    )
  elif hidden == 3:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),
            
      torch.nn.Linear(nodes, out_features)
    )
  elif hidden == 4:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),
      
      torch.nn.Linear(nodes, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.1),

      torch.nn.Linear(nodes, out_features)
    )
  return network

- in_features : 입력데이터의 개수 (x_train.shape : 311 = 300(유전자) + 10(임상변수) + 1(치료유무))
- out_features : 출력노드의 개수

- hidden_layers : 은닉층 수를 가지고 있는 리스트
- number_nodes : 은닉층에 있는 노드 수를 가지고 있는 리스트
- learning_rates : 테스트할 학습률을 가지고 있는 리스트
- brier_scores = brier score을 계산해 append

In [38]:
in_features = x_train.shape[1]
out_features = 1

hidden_layers = [1,2]
number_nodes = [2000, 3000]
learning_rates = [0.0001, 0.001, 0]
brier_scores = []

total_num = len(hidden_layers) * len(number_nodes) * len(learning_rates)
count = 1

for i in hidden_layers:
  for j in number_nodes:
    for k in learning_rates:
      print(count, '/' , total_num)
      net = make_net(in_features, out_features, i, j)
      model = CoxPH(net, tt.optim.Adam)
      batch_size = 639

      if k == 0:
        lrfinder = model.lr_finder(x_train, y_train, batch_size, tolerance = 10)
        model.optimizer.set_lr(lrfinder.get_best_lr())
      else:
        model.optimizer.set_lr(k)
      
      epochs = 512
      callbacks = [tt.callbacks.EarlyStopping()]
      verbose = True

      %%time
      model.fit(x_train, y_train, batch_size, epochs, callbacks, verbose, val_data=val, val_batch_size=batch_size)
      _ = model.compute_baseline_hazards()
      surv = model.predict_surv_df(x_test)

      count += 1

      # calculate ratio
      log_partial_hazard = model.predict(x_for_hr)
      partial_hazard = [np.exp(lph) for lph in log_partial_hazard]

      treat_hr = []
      # ratio with treated and untreated
      for g in range(300):
        treat_hr.append([partial_hazard[g+302]/partial_hazard[g+2],'G' + str(g+1)])
      treat_hr.sort()

      # evaluation
      ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')
      time_grid = np.linspace(durations_test.min(), durations_test.max(), 100)
      score = ev.integrated_brier_score(time_grid)

      if k == 0:
        brier_scores.append([score, i, j, lrfinder.get_best_lr(), treat_hr[:10]])
      else:
        brier_scores.append([score, i, j, k, treat_hr[:10]])

1 / 12
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.72 µs
0:	[0s / 0s],		train_loss: 5.4748,	val_loss: 3.9832
1:	[0s / 0s],		train_loss: 5.4014,	val_loss: 3.9821
2:	[0s / 0s],		train_loss: 5.3513,	val_loss: 3.9808
3:	[0s / 0s],		train_loss: 5.2618,	val_loss: 3.9794
4:	[0s / 0s],		train_loss: 5.2251,	val_loss: 3.9779
5:	[0s / 0s],		train_loss: 5.1673,	val_loss: 3.9761
6:	[0s / 0s],		train_loss: 5.1139,	val_loss: 3.9743
7:	[0s / 0s],		train_loss: 5.0800,	val_loss: 3.9723
8:	[0s / 0s],		train_loss: 5.0425,	val_loss: 3.9703
9:	[0s / 0s],		train_loss: 4.9893,	val_loss: 3.9680
10:	[0s / 0s],		train_loss: 4.9635,	val_loss: 3.9657
11:	[0s / 0s],		train_loss: 4.9261,	val_loss: 3.9632
12:	[0s / 0s],		train_loss: 4.9023,	val_loss: 3.9605
13:	[0s / 0s],		train_loss: 4.8599,	val_loss: 3.9577
14:	[0s / 0s],		train_loss: 4.8345,	val_loss: 3.9547
15:	[0s / 1s],		train_loss: 4.8000,	val_loss: 3.9515
16:	[0s / 1s],		train_loss: 4.7757,	val_loss: 3.9481
17:	[0s / 1s],		train_loss: 4.7420,	va

- brier_score가 가장 작은 것부터 정렬


In [39]:
brier_scores.sort()
selected_genes = []
for i in range(10):
  selected_genes.append(brier_scores[0][4][i][1])

In [40]:
gene_count= {}
for i in brier_scores:
  lst = []
  for j in i[4]:
    lst.append(j[1])
  print(lst)
  for k in lst:
    if k in gene_count.keys():
      gene_count[k] += 1
    else:
      gene_count[k] = 1
sorted(gene_count.items(), key=lambda x: x[1], reverse=True)

['G136', 'G35', 'G60', 'G8', 'G293', 'G123', 'G134', 'G105', 'G132', 'G254']
['G115', 'G77', 'G45', 'G186', 'G89', 'G20', 'G203', 'G150', 'G194', 'G276']
['G292', 'G161', 'G43', 'G149', 'G35', 'G220', 'G293', 'G263', 'G192', 'G280']
['G214', 'G19', 'G211', 'G193', 'G243', 'G140', 'G136', 'G120', 'G105', 'G43']
['G137', 'G35', 'G136', 'G196', 'G149', 'G195', 'G158', 'G11', 'G221', 'G19']
['G29', 'G299', 'G34', 'G161', 'G271', 'G98', 'G295', 'G241', 'G17', 'G19']
['G22', 'G110', 'G192', 'G243', 'G1', 'G260', 'G33', 'G275', 'G149', 'G58']
['G35', 'G243', 'G235', 'G161', 'G34', 'G136', 'G254', 'G185', 'G46', 'G195']
['G163', 'G85', 'G223', 'G164', 'G281', 'G49', 'G212', 'G148', 'G270', 'G159']
['G108', 'G53', 'G150', 'G136', 'G278', 'G235', 'G298', 'G294', 'G35', 'G291']
['G6', 'G231', 'G262', 'G295', 'G64', 'G130', 'G138', 'G69', 'G182', 'G70']
['G234', 'G175', 'G127', 'G55', 'G75', 'G279', 'G262', 'G230', 'G241', 'G135']


[('G136', 5),
 ('G35', 5),
 ('G161', 3),
 ('G149', 3),
 ('G19', 3),
 ('G243', 3),
 ('G293', 2),
 ('G105', 2),
 ('G254', 2),
 ('G150', 2),
 ('G43', 2),
 ('G192', 2),
 ('G195', 2),
 ('G34', 2),
 ('G295', 2),
 ('G241', 2),
 ('G235', 2),
 ('G262', 2),
 ('G60', 1),
 ('G8', 1),
 ('G123', 1),
 ('G134', 1),
 ('G132', 1),
 ('G115', 1),
 ('G77', 1),
 ('G45', 1),
 ('G186', 1),
 ('G89', 1),
 ('G20', 1),
 ('G203', 1),
 ('G194', 1),
 ('G276', 1),
 ('G292', 1),
 ('G220', 1),
 ('G263', 1),
 ('G280', 1),
 ('G214', 1),
 ('G211', 1),
 ('G193', 1),
 ('G140', 1),
 ('G120', 1),
 ('G137', 1),
 ('G196', 1),
 ('G158', 1),
 ('G11', 1),
 ('G221', 1),
 ('G29', 1),
 ('G299', 1),
 ('G271', 1),
 ('G98', 1),
 ('G17', 1),
 ('G22', 1),
 ('G110', 1),
 ('G1', 1),
 ('G260', 1),
 ('G33', 1),
 ('G275', 1),
 ('G58', 1),
 ('G185', 1),
 ('G46', 1),
 ('G163', 1),
 ('G85', 1),
 ('G223', 1),
 ('G164', 1),
 ('G281', 1),
 ('G49', 1),
 ('G212', 1),
 ('G148', 1),
 ('G270', 1),
 ('G159', 1),
 ('G108', 1),
 ('G53', 1),
 ('G278', 1),
 (

- brier_score가 가장 좋은 케이스 출력

In [41]:
print("Brier Score :", brier_scores[0][0])
print("Hidden Layers :", brier_scores[0][1])
print("Number of Nodes :", brier_scores[0][2])
print("Learning Rate :", brier_scores[0][3])
print("Selection :", selected_genes)

Brier Score : 0.06959277772719734
Hidden Layers : 2
Number of Nodes : 2000
Learning Rate : 0.001
Selection : ['G136', 'G35', 'G60', 'G8', 'G293', 'G123', 'G134', 'G105', 'G132', 'G254']


NAN 값을 기준으로 정렬이 끊어진 문제 발견

In [42]:
print("brier_score, hidden layer, number of nodes, learning rate")
for i in brier_scores:
  print(i[:-1])

brier_score, hidden layer, number of nodes, learning rate
[0.06959277772719734, 2, 2000, 0.001]
[0.07188374275409341, 2, 3000, 0.0001]
[0.07222733944452196, 2, 2000, 0.0001]
[0.07223506090124863, 1, 2000, 0.002154434690031894]
[0.07259161577414139, 1, 3000, 0.000849753435908648]
[0.07270541317340544, 1, 3000, 0.001]
[0.07287926555825165, 2, 3000, 0.001]
[0.07327536200020075, 1, 2000, 0.001]
[0.07351248370725726, 1, 3000, 0.0001]
[0.07382761547465361, 2, 3000, 0.0001]
[0.07440251969320119, 2, 2000, 0.0001]
[0.07658301832767403, 1, 2000, 0.0001]
